In [1]:
import datetime
from datetime import timedelta  
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
from tqdm import tqdm 

    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    
    
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis\prolific


In [2]:
data_et = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et.csv')
data_trial = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial.csv')
data_subject = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_subject_raw.csv') \
    .loc[:, 
            ['run_id', 'prolificID', 
             'birthyear', 'webcam_fps', 'webcam_label', 
             'chosenAmount', 'chosenDelay']] \
   .drop_duplicates() \
   .rename(columns={'chosenAmount': 'bonus_USD',
                    'chosenDelay': 'bonus_delay'})
data_subject['prolificID'] = data_subject['prolificID'].str.strip()

data_prolific_int = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/prolific/prolific_export_int.csv') \
    .rename(columns={'participant_id': 'prolificID'}) 

data_prolific_us = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/prolific/prolific_export_us.csv') \
    .rename(columns={'participant_id': 'prolificID'}) \

data_prolific = data_prolific_int \
    .append(data_prolific_us) \
    .merge(data_subject,
           on='prolificID', 
           how='left')

overview = pd.DataFrame(
    [
        [len(data_et)],
        [len(data_trial)],
        [len(data_subject)],
        [len(data_prolific)]
    ], 
    columns=['length'],
    index=['data_et', 
           'data_trial',
           'data_subject', 
           'data_prolific'])
print(overview)

                length
data_et        2700001
data_trial      132631
data_subject       302
data_prolific      357


# Approve subjects

In [3]:
if 'trial_length' in data_prolific.columns: 
    data_prolific = data_prolific.drop(columns='trial_length')

output = []
for subject in data_trial['run_id'].unique():
    prolificID = data_trial.loc[data_trial['run_id']==subject, 'prolificID'].unique()[0]
    trial_length = len(data_trial.loc[data_trial['run_id']==subject, 'trial_index'].unique())
    output.append([subject, prolificID, trial_length])
output = pd.DataFrame(output,
             columns=['run_id', 'prolificID', 'trial_length']
            )

data_prolific = data_prolific.merge(
    output, 
    on=['run_id', 'prolificID'], 
    how='left')

In [4]:
data_et = data_et.merge(
    data_trial.loc[:, ['run_id', 'chinFirst', 'trial_index', 'trial_type', 'task_nr']], 
    on=['run_id', 'trial_index'], 
    how='left'
)

In [ ]:
output = []
for subject in tqdm(data_et['run_id'].unique()):
    m_x_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'x'
    ].mean()
    m_x_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].mean()

    m_y_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'y'
    ].mean()
    m_y_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'y'
    ].mean()

    m_count_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'x'
    ].count()

    m_count_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].count()
    
    n_choseTop = sum(data_trial.loc[
        (data_trial['trial_type']=='eyetracking-choice') &
        (data_trial['run_id']==subject),
        'key_press']==38)

    output.append([subject, m_x_fix, m_y_fix, m_x_choice, m_y_choice, 
                   m_count_fix, m_count_choice, 
                   n_choseTop])

output = pd.DataFrame(output, 
                      columns=['run_id', 'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 
                               'm_count_fix', 'm_count_choice',
                               'n_choseTop'])
output

data_prolific = data_prolific.merge(output, on='run_id', how='left')

 74%|███████████████████████████████████████████████████████████                     | 186/252 [04:29<01:21,  1.23s/it]

# These subjects await review

In [ ]:
data_prolific['status'].unique()

In [ ]:
awaiting_review = data_prolific.loc[
    data_prolific['status']=='AWAITING REVIEW', 
    ['run_id', 'prolificID', 'started_datetime', 'time_taken', 'trial_length',
     #'m_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 'm_count_fix', 'm_count_choice',
     #'n_choseTop'
    ]
].sort_values(by='started_datetime')
awaiting_review
# View(awaiting_review)

In [ ]:
data_prolific.loc[
    data_prolific['status']=='AWAITING REVIEW', 
    'prolificID'].to_csv(header=None, index=False, line_terminator=', ')

In [ ]:
if 'choice_rt' in data_prolific.columns:
    data_prolific = data_prolific.drop(columns='choice_rt')
grouped = data_trial \
    .loc[
        data_trial['trial_type']=='eyetracking-choice', 
        :] \
    .groupby(['prolificID'])['rt'].mean() \
    .reset_index() \
    .rename(columns={'rt': 'choice_rt'})

data_prolific = data_prolific.merge(
    grouped, 
    on='prolificID', 
    how='left')

data_prolific.loc[
    data_prolific['status']=='AWAITING REVIEW', 
    ['run_id', 'prolificID', 'webcam_fps', 
     'choice_rt', 'bonus_USD', 'bonus_delay']
] \
    .sort_values(by='run_id')

# data_pay

In [ ]:
data_pay = data_prolific.loc[
    data_prolific['status']=='APPROVED', 
    [
        'run_id', 'prolificID', 'trial_length', 
        'age',  'Country of Birth', 'Current Country of Residence', 'First Language',
        'Nationality', 'Sex',
        'status', 'reviewed_at_datetime', 'Country of Birth', 'entered_code',
        'session_id', 'started_datetime', 'completed_date_time', 'time_taken',
        'bonus_USD', 'bonus_delay'
    ]
]
print(len(data_pay))

# Bonus payment

Reformat payments

In [ ]:
data_pay['bonus_delay'] = data_pay['bonus_delay'].astype(str)
data_pay['bonus_delay'] = data_pay['bonus_delay'] \
    .replace(['Today', 'Tomorrow', '7 days', 
          '15 days', '30 days', '90 days', 
          '180 days'], 
         [0, 1, 7, 15, 30, 90, 180]) \
    .astype(float)

data_pay['bonus_USD'] = data_pay['bonus_USD'].astype(str)
data_pay['bonus_USD'] = data_pay['bonus_USD'] \
    .replace({'\$':''}, regex = True) \
    .replace('50 cent', 0.5) \
    .astype(float)

data_pay.loc[:, ['bonus_USD', 'bonus_delay']].head(5)

## Missing values

In [ ]:
data_pay.loc[pd.isna(data_pay['bonus_delay']), 
             ['run_id', 'prolificID', 'started_datetime', 
              'trial_length', 'bonus_delay', 'status']]

In [ ]:
# For two subjects, the bonus payment was not recorded
data_pay.loc[
    data_pay['prolificID'].isin([
        '5ef6d07be683903cd5ae171d',
        '5fea6632bf9ae4a79153efdf',
        '60186dc2cc1aa8103499603a',
        '5f4fe72e9468441227166179',
        '5ec5a64c306f255ec98d5cc1',
        '5b8969006651ea000118e42e', 
        '5fb2af792942a58ffe303948', 
        '5edc20443467e28ec4e30f93',
        '5fa1192cf99e161a5cfad1cd', 
        '5d430fdf871f1700017b2a81',
        '5c95970cd676900016e1a940',
    ]), ['bonus_USD', 'bonus_delay']] = [5, 1]

data_pay.loc[data_pay['prolificID'].isin([
        '5fea6632bf9ae4a79153efdf',
        '5ef6d07be683903cd5ae171d',
        '60186dc2cc1aa8103499603a',
        '5f4fe72e9468441227166179',
        '5ec5a64c306f255ec98d5cc1',
        '5b8969006651ea000118e42e', 
        '5fb2af792942a58ffe303948', 
        '5edc20443467e28ec4e30f93',
        '5fa1192cf99e161a5cfad1cd', 
        '5d430fdf871f1700017b2a81',
        '5c95970cd676900016e1a940',
        ]), ['run_id', 'prolificID', 'bonus_USD', 'bonus_delay']]

In [ ]:
data_pay.loc[
    data_pay[['bonus_USD', 'bonus_delay']].isnull().any(axis=1), 
    ['run_id', 'prolificID', 'trial_length', 'started_datetime', 'reviewed_at_datetime']
].sort_values(by='started_datetime')

In [ ]:
data_prolific.loc[
    data_prolific['prolificID']=='5f561a95aa1c4ea13672f138',
]

In [ ]:
data_subject.loc[
    data_subject['prolificID']=='5f561a95aa1c4ea13672f138',
]

## Bonus in other currencies

In [ ]:
data_pay['bonus_GBP'] = data_pay['bonus_USD'] * 0.75
data_pay['bonus_EUR'] = data_pay['bonus_GBP'] * 1.13

## When is the bonus due?

In [ ]:
def add_completed_date(data, data_trial):
    output = []

    for subject in data_trial['run_id'].unique():
        thisSubject = data_trial.loc[data_trial['run_id']==subject] \
            .reset_index(drop=True)
        date_time_obj = datetime.datetime.strptime(
            thisSubject.loc[0, 'recorded_at'], '%Y-%m-%d %H:%M:%S')

        output.append([thisSubject.loc[0, 'run_id'], date_time_obj.date()])
        
    output = pd.DataFrame(output, columns=['run_id', 'date']) 
    
    if 'date' in data.columns: data = data.drop(columns=['date'])
    data = data.merge(output, on='run_id', how='left')
    return data

data_pay = add_completed_date(data_pay, data_trial)
    
data_pay.loc[data_pay['run_id']==444, 'completed_date_time'] = \
    '2021-02-13 21:52:30.000000'
    
data_pay['completed_date']=data_pay.apply(
    lambda x: datetime.datetime.strptime(
        x['completed_date_time'], '%Y-%m-%d %H:%M:%S.%f') \
            .date(),
    axis=1)

data_pay['due_on'] = data_pay['completed_date'] + \
    data_pay['bonus_delay'].map(datetime.timedelta) 

print(len(data_pay))

In [ ]:
data_due = data_pay.loc[:, 
               [
                   'prolificID', 'run_id',
                   'Nationality', 'Current Country of Residence', 'Sex',  
                   'bonus_USD', 'bonus_GBP', 'bonus_EUR', 
                   'completed_date', 'bonus_delay', 'due_on'
               ]
              ]. sort_values(by='due_on')
# View(data_due)

## What bonus is due today? 

In [ ]:
bonus_due_today = data_pay.loc[
    (data_pay['due_on']==datetime.datetime.now().date()), 
    ['run_id', 'prolificID', 'status', 
     'started_datetime', 'reviewed_at_datetime', 
     'bonus_GBP', 'bonus_delay', 'due_on']
]
bonus_due_today['bonus_GBP'] = bonus_due_today['bonus_GBP'].round(2)
bonus_due_today

In [ ]:
bonus_due_today.loc[
    bonus_due_today['run_id']<=130, 
    ['prolificID', 'bonus_GBP']] \
    .to_csv(
        'C:/Users/User/GitHub/WebET_Analysis/prolific/bonusToday_int.csv', 
        index=False, 
        header=False
    )

bonus_due_today.loc[
    bonus_due_today['run_id']>130, 
    ['prolificID', 'bonus_GBP']] \
    .to_csv(
        'C:/Users/User/GitHub/WebET_Analysis/prolific/bonusToday_us.csv', 
        index=False, 
        header=False
    )
print(bonus_due_today['due_on'].unique())

## Bonus left to pay

In [ ]:
data_pay.loc[
    data_pay['due_on']>datetime.datetime.now().date(), 
    ['prolificID', 'completed_date', 'bonus_GBP', 'due_on']]

In [ ]:
left_bonus = data_pay.loc[
    data_pay['due_on']>datetime.datetime.now().date(), 
    'bonus_GBP'].sum()

print(f'Left bonus (GBP): {left_bonus * 1.2 + (left_bonus*0.2*0.2)}')

# Total costs

## Basic payment

In [ ]:
data_pay['basic_GBP'] = 2.25
data_pay.loc[:, ['basic_GBP', 'bonus_GBP']].head(5)

## Total

In [ ]:
print('N =' + str(len(data_pay)) + '\n')

reward_basic = data_pay['basic_GBP'].sum()
service_fee = reward_basic * 0.2
vat = reward_basic * 0.2*0.2
total_basic = data_pay['basic_GBP'].sum() * 1.4
total_bonus = data_pay['bonus_GBP'].sum() * 1.4
total = total_basic + total_bonus

overview = pd.DataFrame(
    [
        [reward_basic], 
        [service_fee], [vat], 
        [total_basic], [total_bonus], [total]
    ], 
    columns=['GBP'],
    index= ['reward_basic', 
        'service_fee', 'vat', 
        'total_basic', 'total_bonus', 'total'])
print(str(overview) + '\n')

print('Total in €, incl. transaction fee: ' + str(total * 1.14))

## Total already paid 
Basic + bonus until today. Use that to compare with prolific bills

In [ ]:
bonus_GBP_already_paid = data_pay.loc[
    data_pay['due_on']<=datetime.datetime.now().date(), 
    'bonus_GBP'].sum()

reward_basic = data_pay['basic_GBP'].sum()
service_fee = (reward_basic + bonus_GBP_already_paid) * 0.2
vat = (reward_basic + bonus_GBP_already_paid) * 0.2*0.2
total_basic = data_pay['basic_GBP'].sum() * 1.4
total_bonus = bonus_GBP_already_paid * 1.4
total = total_basic + total_bonus

overview = pd.DataFrame(
    [
        [reward_basic], 
        [service_fee], [vat], 
        [total_basic], [total_bonus], [total]
    ], 
    columns=['GBP'],
    index= ['reward_basic', 
        'service_fee', 'vat', 
        'total_basic', 'total_bonus', 'total'])
print(str(overview) + '\n')

print('Total in €, incl. transaction fee: ' + str(total * 1.14))

# Prognosis for the larger sample

In [ ]:
def cost_prognosis(n):
    reward_basic = n * data_pay['basic_GBP'].mean()
    reward_bonus = n * data_pay['bonus_GBP'].mean()
        
    total_reward = reward_basic + reward_bonus
    service_fee  = total_reward * 0.2
    vat          = total_reward * 0.2*0.2
    total_basic  = reward_basic + reward_basic*0.2 + reward_basic*0.2*0.2
    total_bonus  = reward_bonus + reward_bonus*0.2 + reward_bonus*0.2*0.2
    total        = total_reward + service_fee + vat

    overview = pd.DataFrame(
        [
            [reward_basic], [reward_bonus], [total_reward],
            [service_fee], [vat], 
            [total_basic], [total_bonus], [total], [total * 1.14]
        ], 
        columns=[n],
        index= ['reward_basic', 'reward_bonus', 'total_reward',
            'service_fee', 'vat', 
            'total_basic', 'total_bonus', 'total GBP', 'total EUR'])
    overview = round(overview, 2)
    return(overview)

print('Budget: ' + str(round(2101*0.88, 2)) + ' GBP / ' + 
      '2101.00 EUR / ' +
      'n=' + str(int(np.floor((2101*0.88)/6.6)))
     )

print('Paid:   1700.50 GBP / ' + 
      str(round(1700.5*1.15, 2)) + ' EUR / ' + 
      'n=' + str(int(np.floor(1700.5/6.6))))
buffer_EUR = 2101-1955.57
buffer_GBP = buffer_EUR * 0.88
print('Buffer: ' + str(round(buffer_GBP, 2)) + ' /  ' + 
      str(round(buffer_EUR, 2)) + ' EUR / n=' + 
      str(int(np.floor(buffer_GBP/6.6))) + '\n')
print('Collect 207 more participants.')

pd.concat([
    cost_prognosis(1),
    cost_prognosis(50),
    cost_prognosis(257), 
    cost_prognosis(207)
], axis=1)

# Export modified prolific data

In [ ]:
data_pay.to_csv('C:/Users/User/GitHub/WebET_Analysis/prolific/data_pay.csv', index=False, header=True)

# Feedback

In [ ]:
print('Success! Script ran through.')